# CPSC 533V: Assignment 3 - Behavioral Cloning and Deep Q Learning

## 48 points total (9% of final grade)

---
This assignment will help you transition from tabular approaches, topic of HW 2, to deep neural network approaches. You will implement the [Atari DQN / Deep Q-Learning](https://arxiv.org/abs/1312.5602) algorithm, which arguably kicked off the modern Deep Reinforcement Learning craze.

In this assignment we will use PyTorch as our deep learning framework.  To familiarize yourself with PyTorch, your first task is to use a behavior cloning (BC) approach to learn a policy.  Behavior cloning is a supervised learning method in which there exists a dataset of expert demonstrations (state-action pairs) and the goal is to learn a policy $\pi$ that mimics this expert.  At any given state, your policy should choose the same action the export would.

Since BC avoids the need to collect data from the policy you are trying to learn, it is relatively simple. 
This makes it a nice stepping stone for implementing DQN. Furthermore, BC is relevant to modern approaches---for example its use as an initialization for systems like [AlphaGo][go] and [AlphaStar][star], which then use RL to further adapte the BC result.  

<!--

I feel like this might be better suited to going lower in the document:

Unfortunately, in many tasks it is impossible to collect good expert demonstrations, making

it's not always possible to have good expert demonstrations for a task in an environemnt and this is where reinforcement learning comes handy. Through the reward signal retrieved by interacting with the environment, the agent learns by itself what is a good policy and can learn to outperform the experts.

-->

Goals:
- Famliarize yourself with PyTorch and its API including models, datasets, dataloaders
- Implement a supervised learning approach (behavioral cloning) to learn a policy.
- Implement the DQN objective and learn a policy through environment interaction.

[go]:  https://deepmind.com/research/case-studies/alphago-the-story-so-far
[star]: https://deepmind.com/blog/article/alphastar-mastering-real-time-strategy-game-starcraft-ii

## Submission information

- Complete the assignment by editing and executing the associated Python files.
- Copy and paste the code and the terminal output requested in the predefined cells on this Jupyter notebook.
- When done, upload the completed Jupyter notebook (ipynb file) on canvas.

## Task 0: Preliminaries

### PyTorch

If you have never used PyTorch before, we recommend you follow this [60 Minutes Blitz][blitz] tutorial from the official website. It should give you enough context to be able to complete the assignment.


**If you have issues, post questions to Piazza**

### Installation

To install all required python packages:

```
python3 -m pip install -r requirements.txt
```

### Debugging


You can include:  `import ipdb; ipdb.set_trace()` in your code and it will drop you to that point in the code, where you can interact with variables and test out expressions.  We recommend this as an effective method to debug the algorithms.


[blitz]: https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html

## Task 1: Behavioral Cloning

Behavioral Cloning is a type of supervised learning in which you are given a dataset of expert demonstrations tuple $(s, a)$ and the goal is to learn a policy function $\hat a = \pi(s)$, such that $\hat a = a$.

The optimization objective is $\min_\theta D(\pi(s), a)$ where $\theta$ are the parameters the policy $\pi$, in our case the weights of a neural network, and where $D$ represents some difference between the actions.

---

Before starting, we suggest reading through the provided files.

For Behavioral Cloning, the important files to understand are: `model.py`, `dataset.py` and `bc.py`.

- The file `model.py` has the skeleton for the model (which you will have to complete in the following questions),

- The file `dataset.py` has the skeleton for the dataset the model is being trained with,

- and, `bc.py` will have all the structure for training the model with the dataset.


### 1.1 Dataset

We provide a pickle file with pre-collected expert demonstrations on CartPole from which to learn the policy $\pi$. The data has been collected from an expert policy on the environment, with the addition of a small amount of gaussian noise to the actions.

The pickle file contains a list of tuples of states and actions in `numpy` in the following way:

```
[(state s, action a), (state s, action a), (state s, action a), ...]
```

In the `dataset.py` file, we provide skeleton code for creating a custom dataset. The provided code shows how to load the file.

Your goal is to overwrite the `__getitem__` function in order to return a dictionary of tensors of the correct type.

Hint: Look in the `bc.py` file to understand how the dataset is used.

Answer the following questions:

- [**QUESTION 2 points]** Insert your code in the placeholder below.

In [5]:
def __getitem__(self, index):
        d = {}
        d["state"] = self.data[index][0]
        d["action"] = self.data[index][1]
        return d

- **[QUESTION 2 points]** How big is the dataset provided?

777.491 KB or 99660 state-action pairs

- **[QUESTION 2 points]** What is the dimensionality of $s$ and what range does each dimension of $s$ span?  I.e., how much of the state space does the expert data cover?
The dimensionality of s is 4

Using the following code:

>>> np.amin(np.asarray([da[0] for da in ddd]), 0)
array([-0.72267057, -0.43303689, -0.05007198, -0.38122098])
>>> np.amax(np.asarray([da[0] for da in ddd]), 0)
array([2.39948596, 1.84697975, 0.14641718, 0.47143314])

the expert data covers from
array([-0.72267057, -0.43303689, -0.05007198, -0.38122098])

to

array([2.39948596, 1.84697975, 0.14641718, 0.47143314])

Recall from a2 that the state space is:
[-4.8, 4.8] X R X [-0.418, 0.418] X R

where R is (-inf, inf)


- **[QUESTION 2 points]** What are the dimensionalities and ranges of the action $a$ in the dataset (how much of the action space does the expert data cover)?


The dimensionality of the actions is 1 and it covers 0 and 1, i.e. all of the action space



### 1.2 Environment

Recall the state and action space of CartPole, from the previous assignment.

- **[QUESTION 2 points]** Considering the full state and action spaces, do you think the provided expert dataset has good coverage?  Why or why not? How might this impact the performance of our cloned policy?

The expert state space seems reasonable if we recall the previous assignment wherein we clipped the second and fourth dimensions to [-2, 2]. The expert data does not span [-2, 2] completely, but it does an okay job. Overall the expert data appears to be biased towards positive values in each dimension, but it doesn't seem unreasonable.

### 1.3 Model

The file `model.py` provides skeleton code for the model. Your goal is to create the architecture of the network by adding layers that map the input to output.

You will need to update the `__init__` method and the `forward` method.

The `select_action` method has already been written for you.  This should be used when running the policy in the environment, while the `forward` function should be used at training time.

- [**QUESTION 5 points]** Insert your code in the placeholder below.

In [4]:
# PLACEHOLDER TO INSERT YOUR MyModel class here

class MyModel(nn.Module):
    def __init__(self, state_size, action_size):
        super(MyModel, self).__init__()
        self.layer = nn.Linear(state_size, action_size, bias=True, dtype=float)
        self.tanh = nn.Tanh()

    def forward(self, x):
        return (self.tanh(-self.layer(state)) > 0).type(torch.int)

#     def select_action(self, state):
#         self.eval()
#         x = self.forward(state)
#         self.train()
#         return x.max(1)[1].view(1, 1).to(torch.long)

Answer the following questions:

- **[QUESTION 2 points]** What is the input of the network?

a state vector of dimensionality 4 whose bounds are within the state space mentioned in my answer above 

- **[QUESTION 2 points]** What is the output?

an action either 1 or 0

### 1.4 Training

The file `bc.py` is the entry point for training your behavioral cloning model. The skeleton and the main components are already there.

The missing parts for you to do are:

- Initializing the model
- Choosing a loss function
- Choosing an optimizer
- Playing with hyperparameters to train your model.

- [**QUESTION 5 points]** Insert your code in the placeholder below.

In [5]:
# PLACEHOLDER FOR YOUR CODE HER
# HOW DID YOU INITIALIZE YOUR MODEL, OPTIMIZER AND LOSS FUNCTIONS? PASTE HERE YOUR FINAL CODE
# NOTE: YOU CAN KEEP THE FOLLOWING LINES COMMENTED OUT, AS RUNNING THIS CELL WILL PROBABLY RESULT IN ERRORS

# STATE_SIZE = 4
# ACTION_SIZE = 2

# model = MyModel(STATE_SIZE, ACTION_SIZE)
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
# loss_function = torch.nn.CrossEntropyLoss()

    # def __init__(self, state_size, action_size):
    #    super(MyModel, self).__init__()

    #     hidden_size = 8
    #     output_size = action_size + 1j
    #     self.neural_network = nn.Sequential(
    #         nn.Linear(state_size, hidden_size, bias=True, dtype=torch.double),
    #        nn.ReLU(),
    #         nn.Linear(hidden_size, output_size, bias=True, dtype=torch.double),
    #     )

    # def forward(self, state):
    #     return self.neural_network(state.type(torch.double)).type(torch.double)


You can run your code by doing:

```
python3 bc.py
```

**During all of this assignment, the code in `eval_policy.py` will be your best friend.** At any time, you can test your model by giving as argument the path to the model weights and the environment name using the following command:

```
python3 eval_policy.py --model-path /path/to/model/weights --env ENV_NAME
````

In [4]:
## PASTE YOUR TERMINAL OUTPUT HERE
# NOTE: TO HAVE LESS LINES PRINTED, YOU CAN SET THE VARIABLE PRINT_INTERVAL TO 5 or 10

# [Test on environment] [epoch 88/100] [score 200.00]
# [epoch   89/100] [iter  137500] [loss 0.00036]
# [epoch   89/100] [iter  138000] [loss 0.01690]
# [epoch   89/100] [iter  138500] [loss 0.00252]
# [epoch   90/100] [iter  139000] [loss 0.03160]
# [epoch   90/100] [iter  139500] [loss 0.01707]
# [epoch   90/100] [iter  140000] [loss 0.04642]
# [Test on environment] [epoch 90/100] [score 200.00]
# [epoch   91/100] [iter  140500] [loss 0.00912]
# [epoch   91/100] [iter  141000] [loss 0.01639]
# [epoch   91/100] [iter  141500] [loss 0.00871]
# [epoch   92/100] [iter  142000] [loss 0.00034]
# [epoch   92/100] [iter  142500] [loss 0.00939]
# [epoch   92/100] [iter  143000] [loss 0.00745]
# [Test on environment] [epoch 92/100] [score 200.00]
# [epoch   93/100] [iter  143500] [loss 0.01605]
# [epoch   93/100] [iter  144000] [loss 0.00761]
# [epoch   93/100] [iter  144500] [loss 0.04447]
# [epoch   94/100] [iter  145000] [loss 0.00252]
# [epoch   94/100] [iter  145500] [loss 0.00035]
# [epoch   94/100] [iter  146000] [loss 0.00768]
# [Test on environment] [epoch 94/100] [score 200.00]
# [epoch   95/100] [iter  146500] [loss 0.01391]
# [epoch   95/100] [iter  147000] [loss 0.00030]
# [epoch   95/100] [iter  147500] [loss 0.00544]
# [epoch   95/100] [iter  148000] [loss 0.01700]
# [epoch   96/100] [iter  148500] [loss 0.00050]
# [epoch   96/100] [iter  149000] [loss 0.00429]
# [epoch   96/100] [iter  149500] [loss 0.04258]
# [Test on environment] [epoch 96/100] [score 200.00]
# [epoch   97/100] [iter  150000] [loss 0.00620]
# [epoch   97/100] [iter  150500] [loss 0.00064]
# [epoch   97/100] [iter  151000] [loss 0.00979]
# [epoch   98/100] [iter  151500] [loss 0.01346]
# [epoch   98/100] [iter  152000] [loss 0.01429]
# [epoch   98/100] [iter  152500] [loss 0.00150]
# [Test on environment] [epoch 98/100] [score 199.30]
# [epoch   99/100] [iter  153000] [loss 0.01569]
# [epoch   99/100] [iter  153500] [loss 0.00455]
# [epoch   99/100] [iter  154000] [loss 0.03336]
# [epoch  100/100] [iter  154500] [loss 0.00014]
# [epoch  100/100] [iter  155000] [loss 0.00361]
# [epoch  100/100] [iter  155500] [loss 0.01070]
# [Test on environment] [epoch 100/100] [score 200.00]
# Saving model as behavioral_cloning_CartPole-v0.pt



**[QUESTION 2 points]** Did you manage to learn a good policy? How consistent is the reward you are getting?

Yes, the reward is consistently around 200, so it has solved the problem.

## Task 2: Deep Q Learning

There are two main issues with the behavior cloning approach.

- First, we are not always lucky enough to have access to a dataset of expert demonstrations.
- Second, replicating an expert policy suffers from compounding error. The policy $\pi$ only sees these "perfect" examples and has no knowledge on how to recover from states not visited by the expert. For this reason, as soon as it is presented with a state that is off the expert trajectory, it will perform poorly and will continue to deviate from a good trajectory without the possibility of recovering from errors.

---
The second task consists in solving the environment from scratch, using RL, and most specifically the DQN algorithm, to learn a policy $\pi$.

For this task, familiarize yourself with the file `dqn.py`. We are going to re-use the file `model.py` for the model you created in the previous task.

Your task is very similar to the one in the previous assignment, to implement the Q-learning algorithm, but in this version, our Q-function is approximated with a neural network.

The algorithm (excerpted from Section 6.5 of [Sutton's book](http://incompleteideas.net/book/RLbook2018.pdf)) is given below:

![DQN algorithm](https://i.imgur.com/Mh4Uxta.png)

### 2.0 Think about your model...



**[QUESTION 2 points]** In DQN, we are using the same model as in task 1 for behavioral cloning. In both tasks the model receives as input the state and in both tasks the model outputs something that has the same dimensionality as the number of actions. These two outputs, though, represent very different things. What is each one representing?

The output of the behavioral cloning is a pair of probabilities of taking action 0 or 1 whereas the output of DQN is the pair of Q-values for action 0 and 1.

### 2.1 Update your Q-function

Complete the `optimize_model` function. This function receives as input a `state`, an `action`, the `next_state`, the `reward` and `done` representing the tuple $(s_t, a_t, s_{t+1}, r_t, done_t)$. Your task is to update your Q-function as shown in the [Atari DQN paper](https://arxiv.org/abs/1312.5602) environment. For now don't be concerned with the experience replay buffer. We'll get to that later.

![Loss function](https://i.imgur.com/tpTsV8m.png)

- [**QUESTION 8 points]** Insert your code in the placeholder below.

In [7]:
def optimize_model(state, action, next_state, reward, done):
    with torch.no_grad():
        next_state = torch.tensor([next_state], device=device, dtype=torch.double)
        target_q = target(next_state).max(1)[0].type(torch.double)

    use_next = torch.tensor(not done, device=device, dtype=torch.bool)
    next_state_action_value = target_q * use_next
    y = (torch.tensor(reward) + GAMMA * next_state_action_value).type(torch.double)

    estimate = model(state)[action]
    loss = l2loss_function(estimate, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


### 2.2 $\epsilon$-greedy strategy

You will need a strategy to explore your environment. The standard strategy is to use $\epsilon$-greedy. Implement it in the `choose_action` function template.

- [**QUESTION 5 points]** Insert your code in the placeholder below.

In [9]:
def choose_action(state, test_mode=False):
    if random.random() < EPS_EXPLORATION:
        return torch.tensor([[random.randrange(n_actions)]], device=device, dtype=torch.long)
    else:
        with torch.no_grad():
            state = torch.tensor([state], device=device, dtype=torch.double)
            return model(state).max(1)[1].view(1, 1).to(torch.long)


### 2.3 Train your model

Try to train a model in this way.

You can run your code by doing:

```
python3 dqn.py
```

**[QUESTION 2 points]** How many episodes does it take to learn (ie. reach a good reward)?

Around 2000-2200 episodes

In [1]:
## PASTE YOUR TERMINAL OUTPUT HERE
# NOTE: TO HAVE LESS LINES PRINTED, YOU CAN SET THE VARIABLE PRINT_INTERVAL TO 5 or 10

[Episode 1914/4000] [Steps  116] [reward 117.0]
[Episode 1915/4000] [Steps  103] [reward 104.0]
[Episode 1916/4000] [Steps  111] [reward 112.0]
[Episode 1917/4000] [Steps  118] [reward 119.0]
[Episode 1918/4000] [Steps  127] [reward 128.0]
[Episode 1919/4000] [Steps  199] [reward 200.0]
[Episode 1920/4000] [Steps  115] [reward 116.0]
[Episode 1921/4000] [Steps  121] [reward 122.0]
[Episode 1922/4000] [Steps  118] [reward 119.0]
[Episode 1923/4000] [Steps  124] [reward 125.0]
[Episode 1924/4000] [Steps  148] [reward 149.0]
[Episode 1925/4000] [Steps  145] [reward 146.0]
...
saving model.
[TEST Episode 2200] [Average Reward 195.3]


### 2.4 Add the Experience Replay Buffer

If you read the DQN paper (and as you can see from the algorithm picture above), the authors make use of an experience replay buffer to learn faster. We provide an implementation in the file `replay_buffer.py`. Update the `train_reinforcement_learning` code to push a tuple to the replay buffer and to sample a batch for the `optimize_model` function.

**[QUESTION 5 points]** How does the replay buffer improve performances?

In [12]:
## PASTE YOUR TERMINAL OUTPUT HERE
# NOTE: TO HAVE LESS LINES PRINTED, YOU CAN SET THE VARIABLE PRINT_INTERVAL TO 5 or 10

## Task 3: Extra

Ideas to experiment with:

- Is $\epsilon$-greedy strategy the best strategy available? Why not trying something different.
- Why not make use of the model you have trained in the behavioral cloning part and fine-tune it with RL? How does that affect performance?
- You are perhaps bored with `CartPole-v0` by now. Another environment we suggest trying is `LunarLander-v2`. It will be harder to learn but with experimentation, you will find the correct optimizations for success. Piazza is also your friend :)
- What about learning from images? This requires more work because you have to extract the image from the environment. However, would it be possible? How much more challenging might you expect the learning to be in this case?
- The ReplayBuffer implementation provided is very simple. In class we have briefly mentioned Prioritized Experience Replay; how would the learning process change?
- An improvement over DQN is DoubleDQN, which is a very simple addition to the current code.



In [13]:
# YOU CAN USE THIS CODEBLOCK AND ADD ANY BLOCK BELOW AS YOU NEED
# TO SHOW US THE IDEAS AND EXTRA EXPERIMENTS YOU RUN.
# HAVE FUN!